# Python Notebook for General Transit Feed Specification Data Analysis
## created by Francis Roberto for IFN 711 Research in Practice Unit

## Research Paper Title:
## Public Transport Data Analytics in Brisbane City: Visualizing Spatial-Temporal Performance Indicators (2021)
### -------------------------------------------------------------------------------------------------------------------------------

## First Assumption of this application is that PostgreSQL has been downloaded and setup in the local machine
### ** For my experimentation, I have installed PGAdmin (a PostgreSQL client with UI) on my system for rapid querying of the data tables. It already includes PostgreSQL.

## 1. Importing Python Libraries and Setting the Dictionary of your PostgreSQL details

In [1]:
# import sys to get more detailed Python exception info
import sys
# import the connect library for psycopg2
import psycopg2
# import pandas for using dataframe
import pandas as pd
# Import glob libraries
import glob

# import the error handling libraries for psycopg2
from psycopg2 import OperationalError, errorcodes, errors
# import StringIO for reading files from your machine
from io import StringIO

# Set the maximum number of rows that pandas datframe will display
pd.set_option('display.max_rows', 200)

In [2]:
# Note: please change your host, username & password as per your own values in this connection parameters
conn_params_dic = {
    "host"      : "localhost",
    "user"      : "postgres",
    "password"  : "Postgres_2016"
}

In [3]:
# a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()
    # get the line number when exception occured
    line_n = traceback.tb_lineno
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type)
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

In [4]:
# a connect function to connect to the PostgreSQL database server
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successful..................")
        
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None
    
    return conn

## 2. Create the PostgreSQL Database for the GTFS data storage
### ATTENTION! Enter the desired name of your database below

In [5]:
# Input the preferred identifier/name of your database in the db_name variable value enclosed in the quotes
db_name = 'brisbane_gtfs'

##  **  ATTENTION! The next cell needs to be done once only hence, it is commented out. Uncomment the body of the code creating the database the first time.

In [6]:
# ## Creates a database named brisbane_gtfs in PostgreSQL database

# #<-----Uncomment lines below----->#

# conn = connect(conn_params_dic)
# conn.autocommit = True
# if conn!=None:
    
#     try:
#         cursor = conn.cursor();
#         # Dropping database brisbane_gtfs if exists
#         cursor.execute(f"DROP DATABASE IF EXISTS {db_name};")
    
#         # Creating a database
#         cursor.execute(f"CREATE DATABASE {db_name};");
#         print(f"{db_name} database is created successfully............")
    
#         # Closing the cursor & connection
#         cursor.close()
#         conn.close()
        
#     except OperationalError as err:
#         # pass exception to function
#         show_psycopg2_exception(err)
#         # set the connection to 'None' in case of error
#         conn = None
# conn.close

In [7]:
# this will update the connection parameters with name of the database you created
conn_params_dic.update({'database':f'{db_name}'})
print(conn_params_dic)

{'host': 'localhost', 'user': 'postgres', 'password': 'Postgres_2016', 'database': 'brisbane_gtfs'}


## 3. Locating each of the files of the GTFS Static in your local machine, load them to individual Python Dataframes
### ATTENTION! The filepaths of each of the GTFS Static file that are needed here are to be replaced
### according to the new locations when working on another machine or folder.

In [8]:
# Locate the path of the GTFS trips.txt and load it to a Python dataframe
trips_df = pd.read_csv(r"/Users/francisroberto/OneDrive - Queensland University of Technology/QUT/IFN712/GTFS DATA/GTFS Static 26-8-2021/trips.txt",index_col=False)

# trips_df.head()

In [9]:
# Locate the path of the GTFS routes.txt and load it to a Python dataframe
routes_df = pd.read_csv(r"/Users/francisroberto/OneDrive - Queensland University of Technology/QUT/IFN712/GTFS DATA/GTFS Static 26-8-2021/routes.txt",index_col=False)

# routes_df.head()

In [10]:
# Locate the path of the GTFS stops.txt and load it to a Python dataframe
stops_df = pd.read_csv(r"/Users/francisroberto/OneDrive - Queensland University of Technology/QUT/IFN712/GTFS DATA/GTFS Static 26-8-2021/stops.txt",index_col=False)

# stops_df.head()

In [11]:
# Locate the path of the GTFS stop_times.txt and load it to a Python dataframe
stop_times_df = pd.read_csv(r"/Users/francisroberto/OneDrive - Queensland University of Technology/QUT/IFN712/GTFS DATA/GTFS Static 26-8-2021/stop_times.txt",index_col=False)

# stop_times_df.head()

In [12]:
# Locate the path of the GTFS shapes.txt and load it to a Python dataframe
shapes_df = pd.read_csv(r"/Users/francisroberto/OneDrive - Queensland University of Technology/QUT/IFN712/GTFS DATA/GTFS Static 26-8-2021/shapes.txt",index_col=False)

# shapes_df.head()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## 4. Creating individual database tables for each of the static files that we have loaded to dataframes then storing those dataframes to those tables

In [13]:
# Saving the dataframes to a list then we run a function to save the items in the list to the database tables later
df_lists = [trips_df, routes_df, stops_df, stop_times_df, shapes_df]


In [14]:
# Create a table named trips in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table trips if exists
        cursor.execute("DROP TABLE IF EXISTS trips;")
        
        sql = '''CREATE TABLE trips(
            route_id VARCHAR,
            service_id VARCHAR,
            trip_id VARCHAR,
            trip_headsign VARCHAR,
            direction_id VARCHAR,
            block_id VARCHAR,
            shape_id VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print("trips table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
trips table is created successfully................


In [15]:
# Create a table named routes in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table trips if exists
        cursor.execute("DROP TABLE IF EXISTS routes;")
        
        sql = '''CREATE TABLE routes(
            route_id VARCHAR,
            route_short_name VARCHAR,
            route_long_name VARCHAR,
            route_desc VARCHAR,
            route_type INTEGER,
            route_url VARCHAR,
            route_color VARCHAR,
            route_text_color VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print("routes table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
routes table is created successfully................


In [16]:
# Create a table named stops in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table stops if exists
        cursor.execute("DROP TABLE IF EXISTS stops;")
        
        sql = '''CREATE TABLE stops(
            stop_id VARCHAR,
            stop_code VARCHAR,
            stop_name VARCHAR,
            stop_desc VARCHAR,
            stop_lat VARCHAR,
            stop_lon VARCHAR,
            zone_id VARCHAR,
            stop_url VARCHAR,
            location_type VARCHAR,
            parent_station VARCHAR,
            platform_code VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print("stops table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
stops table is created successfully................


In [17]:
# Create a table named stop_times in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table stop_times if exists
        cursor.execute("DROP TABLE IF EXISTS stop_times;")
        
        sql = '''CREATE TABLE stop_times(
            trip_id VARCHAR,
            arrival_time VARCHAR,
            departure_time VARCHAR,
            stop_id VARCHAR,
            stop_sequence VARCHAR,
            pickup_type VARCHAR,
            drop_off_type VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print("stop_times table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
stop_times table is created successfully................


In [18]:
# Create a table named shapes in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table shapes if exists
        cursor.execute("DROP TABLE IF EXISTS shapes;")
        
        sql = '''CREATE TABLE shapes(
            shape_id VARCHAR,
            shape_pt_lat VARCHAR,
            shape_pt_lon VARCHAR,
            shape_pt_sequence VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print("shapes table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
shapes table is created successfully................


## 5. Storing the Loaded GTFS Static Files to the their respective tables in the PostgreSQL Database
### Defining a general function to be used for loading and storing the GTFS files in the dataframe
### to their respective tables in the Database

In [19]:
## Define common function using copy_from() with StringIO to store a 
# dataframe to a table in the PostgreSQL Database

def copy_from_dataFile_StringIO(conn, datafrm, table):
    
  # save dataframe to an in memory buffer
    buffer = StringIO()
    datafrm.to_csv(buffer, header=False, index = False)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        sql = f"COPY {table} FROM STDIN CSV"
        cursor.copy_expert(sql, buffer)
        conn.commit()
        print(f"{table}: Data inserted using copy_from_datafile_StringIO() successfully....")
    except (Exception, psycopg2.DatabaseError) as error:
        show_psycopg2_exception(err)
        cursor.close()
    cursor.close()

In [20]:
# Creating a mapping dictionary to map the tables to the dataframes respectively
table_and_map_dic = {
    "trips": trips_df,
    "routes": routes_df,
    "stops": stops_df,
    "stop_times": stop_times_df,
    "shapes": shapes_df
}

### Looping through each dataframes and saving them to their respective tables

In [21]:
# Running the function that stores the each dataframes to their respective tables in the database
for table, dataframe in table_and_map_dic.items():
    conn = connect(conn_params_dic)
    conn.autocommit = True
    copy_from_dataFile_StringIO(conn, dataframe, table)
    print(f"Success {table}")


Connecting to the PostgreSQL...........
Connection successful..................
trips: Data inserted using copy_from_datafile_StringIO() successfully....
Success trips
Connecting to the PostgreSQL...........
Connection successful..................
routes: Data inserted using copy_from_datafile_StringIO() successfully....
Success routes
Connecting to the PostgreSQL...........
Connection successful..................
stops: Data inserted using copy_from_datafile_StringIO() successfully....
Success stops
Connecting to the PostgreSQL...........
Connection successful..................
stop_times: Data inserted using copy_from_datafile_StringIO() successfully....
Success stop_times
Connecting to the PostgreSQL...........
Connection successful..................
shapes: Data inserted using copy_from_datafile_StringIO() successfully....
Success shapes


## Transforming the SHAPES TABLE
## 6. Querying the PostgreSQL Database GTFS Static Shapes Table to transform it to another form of table.
### The new table has the Link Column, Destination Latitude, and Destination Longitude.
### Links are the smaller components of the Shape of the Route. They are the groups of shape point sequences (shape_pt_sequence field of the shape file/table).
### This process is needed in order to visualize the links in Tableau using the MAKELINE() and MAKEPOINT() functions.

In [22]:
# Create a function that takes the shape.txt and creates the table with the derived link and the arrival lat and long
# from a shape_pt_sequence with a lat and long.
# This is automating the process of creating Links between two Stops. It uses groups Shape Pt Sequences into 1 link.
# A whole Shape or Shape ID is a composed of multiple smaller Links.

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Sql query
        generate_link_query = """
            select s.shape_id,
            shape_pt_lat,
            shape_pt_lon,
            s.shape_pt_sequence,
            (s.shape_pt_sequence::INTEGER/10000)::INTEGER as link,
            lead(shape_pt_lat) over (PARTITION BY s.shape_id order by s.shape_pt_sequence::DECIMAL) as destination_lat,
            lead(shape_pt_lon) over (PARTITION BY s.shape_id order by s.shape_pt_sequence::DECIMAL) as destination_lon
            from shapes s
            order by s.shape_id
            """
        
        # Execute Query
        # cursor.execute(generate_link_query)
        links_table = pd.read_sql(generate_link_query, conn);
        print("Route Shape Links was created successfully................")
        links_table.to_csv("shapes_links.csv", index=False)
        print("shapes_links.csv has been created and saved as well................")
        print(links_table)
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
Route Shape Links was created successfully................
shapes_links.csv has been created and saved as well................
         shape_id         shape_pt_lat        shape_pt_lon shape_pt_sequence  \
0         1000001  -27.623727000000002  152.96993600000002             10001   
1         1000001           -27.623545  152.96988000000002             10002   
2         1000001  -27.623196000000004          152.969737             10003   
3         1000001           -27.622804          152.969492             10004   
4         1000001           -27.622804          152.969457             10005   
...           ...                  ...                 ...               ...   
1611857  VLDB0057  -27.430971000000003          153.070724            240013   
1611858  VLDB0057  -27.431054999999997          153.071827            240014   
1611859  VLDB0057  -27.431165000000004          153.072732            240

In [23]:
# Inspecting the Links Table
links_table.head(20)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,link,destination_lat,destination_lon
0,1000001,-27.623727000000002,152.96993600000002,10001,1,-27.623545,152.96988000000002
1,1000001,-27.623545,152.96988000000002,10002,1,-27.623196000000004,152.969737
2,1000001,-27.623196000000004,152.969737,10003,1,-27.622804,152.969492
3,1000001,-27.622804,152.969492,10004,1,-27.622804,152.969457
4,1000001,-27.622804,152.969457,10005,1,-27.622798,152.969423
5,1000001,-27.622798,152.969423,10006,1,-27.622784999999997,152.969391
6,1000001,-27.622784999999997,152.969391,10007,1,-27.622767,152.969363
7,1000001,-27.622767,152.969363,10008,1,-27.622744,152.96934
8,1000001,-27.622744,152.96934,10009,1,-27.622736,152.969335
9,1000001,-27.622736,152.969335,10010,1,-27.622718,152.969322


## 7. Creating a table in the PostgreSQL Database where the transformed Shape File is going to be stored.
### The Table is named "shapes_links"

In [24]:
# Create a table named shapes_links in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table shapes if exists
        cursor.execute("DROP TABLE IF EXISTS shapes_links;")
        
        sql = '''CREATE TABLE shapes_links(
            shape_id VARCHAR,
            shape_pt_lat VARCHAR,
            shape_pt_lON VARCHAR,
            shape_pt_sequence VARCHAR,
            link VARCHAR,
            destination_lat VARCHAR,
            destination_lon VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print("shapes_links table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
shapes_links table is created successfully................


In [25]:
# # This cell is collapsed and commented out
#  
# # Define function using copy_from() with StringIO to insert the dataframe
# def copy_from_dataFile_StringIO(conn, datafrm, table):
#     """ Connect to the PostgreSQL database server """
    
       
#   # save dataframe to an in memory buffer
#     buffer = StringIO()
#     datafrm.to_csv(buffer, header=False, index = False)
#     buffer.seek(0)
    
#     cursor = conn.cursor()

#     try:   
#         # Clear the table from old records
#         cursor.execute(f"TRUNCATE TABLE  {table};");    

#         # Check the records before copying
#         cursor.execute(f"Select count(*) from {table}");
#         record = cursor.fetchall()
#         print("QUERY RESULT BEFORE ", record, "\n")
        
#         cursor.copy_from(buffer, table, sep=",")
#         conn.commit()
#         print("Data inserted using copy_from_datafile_StringIO() successfully....")

        
#         # Check the records after copying
#         cursor.execute(f"Select count(*) from {table}");
#         record = cursor.fetchall()
#         print("QUERY RESULT AFTER ", record, "\n")
        
#     except (Exception, psycopg2.DatabaseError) as err:
#         # pass exception to function
#         show_psycopg2_exception(err)
#         # print(err)
#     finally:
#         if conn is not None:
#             cursor.close()
#             conn.close()
#             print('Database connection closed.')

In [26]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
print('adding data to shapes_links table')

# Run the copy_from_dataFile_StringIO() method
copy_from_dataFile_StringIO(conn, links_table, 'shapes_links')

Connecting to the PostgreSQL...........
Connection successful..................
adding data to shapes_links table
shapes_links: Data inserted using copy_from_datafile_StringIO() successfully....


## 8. ATTENTION! Indicate the Route and Direction to be used for filtering the large GTFS Realtime data
## ** Pre-conditions: Data for 1 day is shared and downloaded to local machine. This data is in the form of multiple CSV files of Trip Updates located inside a \"TU<dd-mm-yyyy>\" folder

In [27]:
# Enter Route ID and Direction ID in these variables' values
# Direction 0 is Inbound and 1 for Outbound

s_route_id = '333'
s_direction_id = '0'

In [28]:
# Query the Static files table in PostGres to get all the listed Shape ID's for 
# the specific route and direction

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Sql query
        get_route_shape_ids_query = f"""
            select distinct(s.shape_id), t.direction_id, t.route_id
            from routes r
            inner join trips t on r.route_id = t.route_id
            inner join shapes s on s.shape_id = t.shape_id
            where r.route_id like '{s_route_id}%'
            and t.direction_id = '{s_direction_id}'
            """
        
        # Execute Query
        # cursor.execute(generate_link_query)
        shape_ids_df = pd.read_sql(get_route_shape_ids_query, conn);
        print("Route Shape ID's were queries successfully................")
        print(shape_ids_df)
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
Route Shape ID's were queries successfully................
  shape_id direction_id  route_id
0  3330058            0  333-1886
1  3330071            0  333-1886
2  3330072            0  333-1886
3  3330060            0  333-1886


## Displaying the List of Shape ID's and their direction in a particular route
## ** Direction 0 is Inbound and 1 for Outbound

In [29]:
# Display the dataframe of Shape ID, Direction ID and Route ID
shape_ids_df.head(200)

,shape_id,direction_id,route_id
0,3330058,0,333-1886
1,3330071,0,333-1886
2,3330072,0,333-1886
3,3330060,0,333-1886


## 9. ATTENTION! Set the Shape ID you will use for filtering the real-time data by selecting an index from the above resulting table and using that for filtering the GTFS Real-time data
## Direction 0 is Inbound and 1 for Outbound
## Assumption: We know that the Realtime data only contains Shape ID 3330071 for the Route 333 we want to extract.

In [30]:
# Set the shape_id_index variable with your chosen shape_id index
# We select shape_id 3330071 so the index is 2

shape_id_index = 1

shape_ids = tuple(shape_ids_df.iloc[:,0].unique())
print(shape_ids[shape_id_index])
print(type(shape_ids[shape_id_index]))

3330071
<class 'str'>


In [31]:
# Get the common route_id

route_id = (shape_ids_df.iloc[:,2].unique())[0]
print(route_id)
print(type(route_id))

333-1886
<class 'str'>


## 10. Loading and Filtering the Real-time data and storing it in a DataFrame

In [32]:
# Filepaths of three datasets available listed here.
# For the experimental visualization works in Tableau, the filepaths that was used was the dataset with the 26-8-2021 date

# Only Chosen the TU feeds 2-8-2021 in this section to avoid overwriting the 26-8-2021 data in the database
# as I have used the TU feeds 26-8-2021 data for the Tableau Visualization

tu_source_path = r'/Users/francisroberto/OneDrive - Queensland University of Technology/QUT/IFN712/GTFS DATA/TU feeds 2-8-2021'
# tu_source_path = r'/Users/francisroberto/OneDrive - Queensland University of Technology/QUT/IFN712/GTFS DATA/TU feeds 3-8-2021'
# tu_source_path = r'/Users/francisroberto/OneDrive - Queensland University of Technology/QUT/IFN712/GTFS DATA/TU feeds 26-8-2021'

# Locate all required csv files from the source path and save the collection to a variable
tu_csv_files = glob.glob(tu_source_path + "/*.csv")

In [33]:
# Filenames to be used when saving the Filtered_Routes
# filename = f'rt_shape_id_{shape_ids[shape_id_index]}'

In [34]:
# Instantiate an empty DataFrame object
result_df = pd.DataFrame()

# Loop through all the files in the files collection variable and then look for
# shape_id that are included in our shape id list for a specific routes
for file in tu_csv_files:
    df = pd.read_csv(file, usecols=range(1,17), dtype='unicode', low_memory=False)
    newdf = df[(df.shape_id == shape_ids[shape_id_index])]
    
    result_df = result_df.append(newdf, ignore_index=True)

In [35]:
# Inspecting the result_df dataframe that contains the filtered real-time data
result_df.head()

,trip_id,start_time,start_date,route_id,stop_id,stop_sequence,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,id,timestamp,shape_id
0,17517727-BT 21_22-JUL_FUL-Weekday-01,22:45:00,20210802,333-1886,11270,4.0,3.0,1627908843.0,30.0,38.0,1627908877.0,30.0,0.0,TU-17517727-BT 21_22-JUL_FUL-Weekday-01,1627908828,3330071
1,17517727-BT 21_22-JUL_FUL-Weekday-01,22:45:00,20210802,333-1886,11272,5.0,-20.0,1627908880.0,30.0,8.0,1627908907.0,30.0,0.0,TU-17517727-BT 21_22-JUL_FUL-Weekday-01,1627908828,3330071
2,17517727-BT 21_22-JUL_FUL-Weekday-01,22:45:00,20210802,333-1886,469,6.0,25.0,1627909105.0,30.0,25.0,1627909105.0,30.0,0.0,TU-17517727-BT 21_22-JUL_FUL-Weekday-01,1627908828,3330071
3,17517727-BT 21_22-JUL_FUL-Weekday-01,22:45:00,20210802,333-1886,897,7.0,-11.0,1627909249.0,30.0,-11.0,1627909249.0,30.0,0.0,TU-17517727-BT 21_22-JUL_FUL-Weekday-01,1627908828,3330071
4,17517727-BT 21_22-JUL_FUL-Weekday-01,22:45:00,20210802,333-1886,891,8.0,20.0,1627909340.0,30.0,20.0,1627909340.0,30.0,0.0,TU-17517727-BT 21_22-JUL_FUL-Weekday-01,1627908828,3330071


In [36]:
# Listing down and Inspecting the datatypes of the fields result_df dataframe that contains the filtered real-time data
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76070 entries, 0 to 76069
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   trip_id                76070 non-null  object
 1   start_time             76070 non-null  object
 2   start_date             76070 non-null  object
 3   route_id               76070 non-null  object
 4   stop_id                76070 non-null  object
 5   stop_sequence          76070 non-null  object
 6   arrival_delay          72698 non-null  object
 7   arrival_time           72698 non-null  object
 8   arrival_uncertainty    72698 non-null  object
 9   departure_delay        75957 non-null  object
 10  departure_time         75957 non-null  object
 11  departure_uncertainty  75957 non-null  object
 12  schedule_relationship  76070 non-null  object
 13  id                     76070 non-null  object
 14  timestamp              76070 non-null  object
 15  shape_id           

## 11. Creating a table in the PostgreSQL Database to store the filtered GTFS Realtime data
### The table will named filtered_rt_shape_id_3330071_20210802

In [37]:
# Create a table named filtered_rt_shape_id{shape_id} in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table shapes if exists
        cursor.execute(f"DROP TABLE IF EXISTS filtered_rt_shape_id_{shape_ids[shape_id_index]}_20210802;")
        
        sql = f'''CREATE TABLE filtered_rt_shape_id_{shape_ids[shape_id_index]}_20210802(
            trip_id VARCHAR, 
            start_time VARCHAR,
            start_date VARCHAR,
            route_id VARCHAR,
            stop_id VARCHAR,
            stop_sequence VARCHAR,
            arrival_delay VARCHAR,
            arrival_time VARCHAR,
            arrival_uncertainty VARCHAR,
            departure_delay VARCHAR,
            departure_time VARCHAR,
            departure_uncertainty VARCHAR,
            schedule_relationship VARCHAR,
            id VARCHAR,
            timestamp VARCHAR,
            shape_id VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print(f"filtered_rt_shape_id_{shape_ids[shape_id_index]}_20210802 table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
filtered_rt_shape_id_3330071_20210802 table is created successfully................


In [38]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
print(f'adding data to filtered_rt_shape_id_{shape_ids[shape_id_index]}_20210802 table')

# Run the copy_from_dataFile_StringIO() method to save the filtered real-time
# data to the created filtered_rt_shape_id_{shape_ids[shape_id_index]} table
copy_from_dataFile_StringIO(conn, result_df, f'filtered_rt_shape_id_{shape_ids[shape_id_index]}_20210802')

Connecting to the PostgreSQL...........
Connection successful..................
adding data to filtered_rt_shape_id_3330071_20210802 table
filtered_rt_shape_id_3330071_20210802: Data inserted using copy_from_datafile_StringIO() successfully....


<!-- # Create a table named filtered_rt_shape_id{shape_id} in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table shapes if exists
        cursor.execute(f"DROP TABLE IF EXISTS filtered_rt_shape_id_{shape_ids[shape_id_index]};")
        
        sql = f'''CREATE TABLE filtered_rt_shape_id_{shape_ids[shape_id_index]}(
            trip_id VARCHAR, 
            start_time VARCHAR,
            start_date VARCHAR,
            route_id VARCHAR,
            stop_id VARCHAR,
            stop_sequence VARCHAR,
            arrival_delay VARCHAR,
            arrival_time VARCHAR,
            arrival_uncertainty VARCHAR,
            departure_delay VARCHAR,
            departure_time VARCHAR,
            departure_uncertainty VARCHAR,
            schedule_relationship VARCHAR,
            id VARCHAR,
            timestamp VARCHAR,
            shape_id VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print(f"filtered_rt_shape_id_{shape_ids[shape_id_index]} table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None -->

In [39]:
# # Connect to the database
# conn = connect(conn_params_dic)
# conn.autocommit = True
# print(f'adding data to filtered_rt_shape_id_{shape_ids[shape_id_index]} table')
# # Run the copy_from_dataFile_StringIO() method to save the filtered real-time
# # data to the created filtered_rt_shape_id_{shape_ids[shape_id_index]} table
# copy_from_dataFile_StringIO(conn, result_df, f'filtered_rt_shape_id_{shape_ids[shape_id_index]}')

In [40]:
# # Filenames to be used when saving the Filtered_Routes
# filename1 = f'rt_route_id_{route_id}'
# # Instantiate an empty DataFrame object
# result_df1 = pd.DataFrame()

# # Loop through all the files in the files collection variable and then look for shape_id that are included in our shape id list for a specific routes
# for file in tu_csv_files:
#     df1 = pd.read_csv(file, usecols=range(1,17))
#     newdf1 = df1[(df1.route_id == route_id)]
#     result_df1 = result_df1.append(newdf, ignore_index=True)
# result_df1.head()

In [41]:
# result_df1.info()
# result_df1.shape_id.value_counts()

In [42]:
# result_df1.shape_id.unique()

In [43]:
# print(f"rt_route_id_{route_id}")
route_id_formatted = route_id.replace("-", "_")
# print(route_id_formatted)

In [44]:
# # Create a table named filtered_rt_shape_id{shape_id} in the brisbane_gtfs PostgreSQL database

# conn = connect(conn_params_dic)
# conn.autocommit = True
# if conn!=None:
    
#     try:
#         cursor = conn.cursor();
#         # Dropping table shapes if exists
#         cursor.execute(f"DROP TABLE IF EXISTS rt_route_id_{route_id_formatted};")
        
#         sql = f'''CREATE TABLE rt_route_id_{route_id_formatted}(
#             trip_id VARCHAR, 
#             start_time VARCHAR,
#             start_date VARCHAR,
#             route_id VARCHAR,
#             stop_id VARCHAR,
#             stop_sequence VARCHAR,
#             arrival_delay VARCHAR,
#             arrival_time VARCHAR,
#             arrival_uncertainty VARCHAR,
#             departure_delay VARCHAR,
#             departure_time VARCHAR,
#             departure_uncertainty VARCHAR,
#             schedule_relationship VARCHAR,
#             id VARCHAR,
#             timestamp VARCHAR,
#             shape_id VARCHAR
#         )'''
        
#         # Creating a table
#         cursor.execute(sql);
#         print(f"rt_route_id_{route_id_formatted} table is created successfully................")
    
#         # Closing the cursor & connection
#         cursor.close()
#         conn.close()
        
#     except OperationalError as err:
#         # pass exception to function
#         show_psycopg2_exception(err)
#         # set the connection to 'None' in case of error
#         conn = None

In [45]:
# # Connect to the database
# conn = connect(conn_params_dic)
# conn.autocommit = True
# print(f'adding data to rt_route_id_{route_id_formatted}')
# # Run the copy_from_dataFile_StringIO() method to save the filtered real-time
# # data to the created rt_route_id_{route_id_formatted} table
# copy_from_dataFile_StringIO(conn, result_df1, f'rt_route_id_{route_id_formatted}')

In [46]:
# Query the filtered real-time data to derive actual travel time and target travel time

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Sql query
        get_travel_time_query = rf"""
SELECT fr.trip_id, fr.start_time, fr.start_date, fr.route_id, fr.stop_sequence,
			fr.arrival_delay, fr.arrival_uncertainty,
			fr.departure_delay, fr.departure_uncertainty, fr.schedule_relationship,
			fr.id, fr.timestamp, fr.shape_id,
			st.departure_time, st.arrival_time, previous_departure,
			actual_travel_time, timestamp_dt, departure_time_dt, arrival_time_dt, previous_departure_dt,
            st.stop_id, st.pickup_type, st.drop_off_type, 
			s.stop_code, s.stop_name, s.stop_desc, s.stop_lat, s.stop_lon, s.zone_id, s.stop_url, 
            s.location_type, s.parent_station, s.platform_code,
			sl.shape_pt_lat, sl.shape_pt_lon, sl.shape_pt_sequence, 
            sl.link, sl.destination_lat, sl.destination_lon
FROM (
	Select *,
	lag(departure_time) over (PARTITION BY trip_id order by stop_sequence) as previous_departure,
	arrival_time - lag(departure_time) over (PARTITION BY trip_id order by stop_sequence) as actual_travel_time,
	TO_TIMESTAMP(timestamp) as timestamp_dt,
	TO_TIMESTAMP(departure_time) as departure_time_dt,
	TO_TIMESTAMP(arrival_time) as arrival_time_dt,
	lag(TO_TIMESTAMP(departure_time)) over (PARTITION BY trip_id order by stop_sequence) as previous_departure_dt
	FROM(
		Select *
		FROM(
			select 
			*,
			ROW_NUMBER() OVER (PARTITION BY trip_id, stop_sequence order by timestamp desc ) as row_number
			from
			(
				select 
				DISTINCT 
				trip_id,
				stop_sequence::DECIMAL as stop_sequence,
				timestamp::DECIMAL,
				CASE WHEN coalesce(departure_time, '') = '' THEN NULL ELSE departure_time::DECIMAL END as departure_time,
				CASE WHEN coalesce(arrival_time, '') = '' THEN NULL ELSE arrival_time::DECIMAL END as arrival_time,
				start_time,
				start_date, 
				route_id, stop_id,  arrival_delay,  arrival_uncertainty, departure_delay, departure_uncertainty, 
				schedule_relationship, id, shape_id

				from public.filtered_rt_shape_id_3330071
			
			) filtered_routes_unique
		
		) filtered_routes_unique
		WHERE row_number = 1
	)filtered_routes
) fr
inner join stop_times st on fr.stop_sequence = st.stop_sequence::DECIMAL and fr.trip_id = st.trip_id and  fr.stop_id::DECIMAL = st.stop_id::DECIMAL
inner join stops s on fr.stop_id::DECIMAL = (CASE WHEN s.stop_id~E'^\\d+$' THEN s.stop_id::DECIMAL ELSE 0 END)
inner join shapes_links sl on fr.shape_id = sl.shape_id
            """
        
        # Execute Query
        # cursor.execute(generate_link_query)
        travel_time_df = pd.read_sql(get_travel_time_query, conn);
        print("Links Travel Time table were derived successfully................")
        print(travel_time_df)
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
Links Travel Time table were derived successfully................
                                     trip_id start_time start_date  route_id  \
0       18312543-BT 21_22-AUG_FUL-Weekday-01   06:25:00   20210826  333-1886   
1       18312543-BT 21_22-AUG_FUL-Weekday-01   06:25:00   20210826  333-1886   
2       18312543-BT 21_22-AUG_FUL-Weekday-01   06:25:00   20210826  333-1886   
3       18312468-BT 21_22-AUG_FUL-Weekday-01   06:15:00   20210826  333-1886   
4       18312468-BT 21_22-AUG_FUL-Weekday-01   06:15:00   20210826  333-1886   
...                                      ...        ...        ...       ...   
477166  18321697-BT 21_22-AUG_FUL-Weekday-01   22:15:00   20210826  333-1886   
477167  18321697-BT 21_22-AUG_FUL-Weekday-01   22:15:00   20210826  333-1886   
477168  18321697-BT 21_22-AUG_FUL-Weekday-01   22:15:00   20210826  333-1886   
477169  18321697-BT 21_22-AUG_FUL-Weekday-01   22:15:0

In [47]:
travel_time_df.columns

Index(['trip_id', 'start_time', 'start_date', 'route_id', 'stop_sequence',
       'arrival_delay', 'arrival_uncertainty', 'departure_delay',
       'departure_uncertainty', 'schedule_relationship', 'id', 'timestamp',
       'shape_id', 'departure_time', 'arrival_time', 'previous_departure',
       'actual_travel_time', 'timestamp_dt', 'departure_time_dt',
       'arrival_time_dt', 'previous_departure_dt', 'stop_id', 'pickup_type',
       'drop_off_type', 'stop_code', 'stop_name', 'stop_desc', 'stop_lat',
       'stop_lon', 'zone_id', 'stop_url', 'location_type', 'parent_station',
       'platform_code', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence',
       'link', 'destination_lat', 'destination_lon'],
      dtype='object')

In [48]:
travel_time_df = travel_time_df.loc[:,~travel_time_df.columns.duplicated()]


In [49]:
travel_time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477171 entries, 0 to 477170
Data columns (total 40 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   trip_id                477171 non-null  object             
 1   start_time             477171 non-null  object             
 2   start_date             477171 non-null  object             
 3   route_id               477171 non-null  object             
 4   stop_sequence          477171 non-null  float64            
 5   arrival_delay          477171 non-null  object             
 6   arrival_uncertainty    477171 non-null  object             
 7   departure_delay        477171 non-null  object             
 8   departure_uncertainty  477171 non-null  object             
 9   schedule_relationship  477171 non-null  object             
 10  id                     477171 non-null  object             
 11  timestamp              477171 non-null 

## FINAL DENORMALIZED TRAVEL TIMES CSV GENERATED HERE FOR TABLEAU USE

In [50]:
travel_time_df.to_csv(f'travel_time_{route_id_formatted}_{shape_ids[shape_id_index]}.csv', index=False)

In [51]:
# Create a table named travel_time_{route_id}_{shape_id} in the brisbane_gtfs PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping table shapes if exists
        cursor.execute(f"DROP TABLE IF EXISTS travel_time_{route_id_formatted}_{shape_ids[shape_id_index]};")
        
        sql = rf'''CREATE TABLE travel_time_{route_id_formatted}_{shape_ids[shape_id_index]}(
            trip_id VARCHAR, 
            start_time VARCHAR, 
            start_date VARCHAR, 
            route_id VARCHAR,
            stop_sequence VARCHAR,
			arrival_delay VARCHAR, 
            arrival_uncertainty VARCHAR,
			departure_delay VARCHAR, 
            departure_uncertainty VARCHAR, 
            schedule_relationship VARCHAR,
			id VARCHAR, 
            timestamp VARCHAR, 
            shape_id VARCHAR,
			departure_time VARCHAR, 
            arrival_time VARCHAR, 
            previous_departure VARCHAR,
			actual_travel_time VARCHAR,
            timestamp_dt VARCHAR, 
            departure_time_dt VARCHAR,
            arrival_time_dt VARCHAR, 
            previous_departure_dt VARCHAR,
            stop_id VARCHAR, 
            pickup_type VARCHAR, 
            drop_off_type VARCHAR, 
			stop_code VARCHAR, 
            stop_name VARCHAR, 
            stop_desc VARCHAR, 
            stop_lat VARCHAR, 
            stop_lon VARCHAR, 
            zone_id VARCHAR, 
            stop_url VARCHAR, 
            location_type VARCHAR, 
            parent_station VARCHAR, 
            platform_code VARCHAR,
			shape_pt_lat VARCHAR, 
            shape_pt_lon VARCHAR, 
            shape_pt_sequence VARCHAR, 
            link VARCHAR, 
            destination_lat VARCHAR, 
            destination_lon VARCHAR
        )'''
        
        # Creating a table
        cursor.execute(sql);
        print(f"travel_time_{route_id_formatted}_{shape_ids[shape_id_index]} table is created successfully................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
travel_time_333_1886_3330071 table is created successfully................


In [52]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
print(f'travel_time_{route_id_formatted}_{shape_ids[shape_id_index]}')
# Run the copy_from_dataFile_StringIO() method to save the filtered real-time
# data to the created travel_time_{route_id}_{shape_ids[shape_id_index]}
copy_from_dataFile_StringIO(conn, travel_time_df, f'travel_time_{route_id_formatted}_{shape_ids[shape_id_index]}')

Connecting to the PostgreSQL...........
Connection successful..................
travel_time_333_1886_3330071
travel_time_333_1886_3330071: Data inserted using copy_from_datafile_StringIO() successfully....
